## Causal Query Analysis System (ChromaDB + Gemini)

This notebook implements a causal analysis pipeline for customer service conversations.

**Workflow:**
- Loads turn-level and conversation-level transcript data
- Retrieves semantically similar evidence from a persistent ChromaDB vector store
- Expands user queries and classifies query categories using Gemini
- Generates concise causal explanations with cited transcript evidence
- Outputs structured results (Query ID, Category, System Output, Remarks) to CSV

**Key Technologies:**
- Sentence Transformers for semantic retrieval
- ChromaDB for persistent vector search
- Gemini LLM for intent expansion, classification, and explanation

**Output:**
- `task1_queries.csv` containing analysis-ready results for reporting and PDF export


In [ ]:
import re
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
from google import genai
from google.colab import userdata
import json
import os

turn_df = pd.read_csv("utterances_final.csv")
conv_summary_df = pd.read_csv("conversation_level_summary.csv")

embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="multi-qa-mpnet-base-dot-v1",
    device="cuda"
)

chroma_client = chromadb.PersistentClient(path="chroma_db_turns3")
evidence_collection = chroma_client.get_collection(
    name="evidence_turns",
    embedding_function=embedding_func
)

class CausalQuerySystem:
    def __init__(self, turn_df, conv_summary_df, collection, api_key):
        self.turn_df = turn_df
        self.conv_summary_df = conv_summary_df
        self.collection = collection
        self.client = genai.Client(api_key=api_key)

    def _expand_query_intent(self, user_query):


        prompt = f"""
        You are an expert search optimizer.
        The user is asking: "{user_query}"

        Write a hypothetical customer service dialogue turn (1-2 sentences) that would appear in a transcript related to this issue.
        Include keywords like 'frustrated', 'supervisor', 'delivery', 'fraud', or specific error codes if relevant.
        Focus on the root cause.
        """
        try:
            response = self.client.models.generate_content(
                model='gemini-2.5-flash',
                contents=prompt
            )
            expanded_text = response.text.strip()
            print(f"Query Expanded to: {expanded_text[:100]}...")
            return expanded_text
        except:
            return user_query

    def _get_category(self, user_query):



        prompt = f"""
        Classify this customer service query into ONE of these categories:
        - Escalation Analysis
        - Fraud & Security
        - Logistics & Delivery
        - Billing & Payments
        - General Support

        QUERY: "{user_query}"

        Output only the category name.
        """
        try:
            response = self.client.models.generate_content(
                model='gemini-2.5-flash',
                contents=prompt
            )
            return response.text.strip()
        except:
            return "Causal Analysis"

    def query(self, user_query: str, top_k: int = 9):
        print(f"\n{'='*70}")
        print(f"🔍 CAUSAL QUERY: {user_query}")
        print(f"{'='*70}")

        search_query = self._expand_query_intent(user_query)

        results = self.collection.query(
            query_texts=[search_query],
            n_results=top_k
        )

        retrieved_ids = results['ids'][0]
        retrieved_metas = results['metadatas'][0]
        retrieved_distances = results['distances'][0]

        print(f"✓ Found {len(retrieved_ids)} specific evidence points.")

        evidence_list = []
        unique_transcripts = set()

        for i, full_id in enumerate(retrieved_ids):
            tid, turn_no = full_id.split('_')
            turn_no = int(turn_no)
            unique_transcripts.add(tid)

            meta = retrieved_metas[i]

            try:
                turn_row = self.turn_df[
                    (self.turn_df['transcript_id'] == tid) &
                    (self.turn_df['turn_no'] == turn_no)
                ].iloc[0]
                text_content = turn_row['text']
                speaker = turn_row['speaker']
            except:
                text_content = "Text not found"
                speaker = "Unknown"

            evidence_list.append({
                'transcript_id': tid,
                'turn_no': turn_no,
                'speaker': speaker,
                'text': text_content,
                'outcome': meta.get('outcome', 'Unknown'),
                'sentiment': meta.get('sentiment', 0.0),
                'patterns': meta.get('patterns', '[]'),
                'similarity': round(1 - retrieved_distances[i], 4)
            })

        context_docs = []
        for tid in list(unique_transcripts)[:3]:
            try:
                summary = self.conv_summary_df[self.conv_summary_df['transcript_id'] == tid].iloc[0]


                turns = self.turn_df[self.turn_df['transcript_id'] == tid].sort_values('turn_no')
                turn_text = "\n".join([f"T{t['turn_no']} ({t['speaker']}): {t['text'][:100]}" for _, t in turns.head(20).iterrows()])

                context_docs.append(f"""
                TRANSCRIPT {tid}:
                - Outcome: {summary['outcome']}
                - Reason: {summary['reason_for_call']}
                - Excerpt:
                {turn_text}
                """)
            except:
                continue

        # 4. Generate Causal Explanation
        print(f"✓ Generating explanation with Gemini...")
        explanation = self._generate_gemini_response(user_query, evidence_list, context_docs)

        return {
            'query': user_query,
            'explanation': explanation,
            'evidence_count': len(evidence_list),
            'retrieved_ids': list(unique_transcripts)
        }

    def _generate_gemini_response(self, query, evidence, context):
        # Format the evidence list for the prompt
        evidence_str = ""
        for i, ev in enumerate(evidence, 1):
            evidence_str += f"""
            {i}. Transcript {ev['transcript_id']} (Turn {ev['turn_no']})
               - Speaker: {ev['speaker']}
               - Text: "{ev['text']}"
               - Sentiment: {ev['sentiment']}
               - Detected Patterns: {ev['patterns']}
            """

        context_str = "\n".join(context)

        prompt = f"""
You are an expert Causal Analysis AI. Explain WHY specific outcomes occurred using the provided evidence.

USER QUERY: "{query}"

RETRIEVED CAUSAL EVIDENCE:
{evidence_str}

CONVERSATION CONTEXT:
{context_str}

INSTRUCTIONS:
1. **ULTRA-BREVITY:** Total response MUST be under 100 words. Use basic, simple English.
2. **STRICT QUANTITY:** You MUST provide EXACTLY 4 Causal Factors and EXACTLY 4 Recommendations.
3. **TELEGRAPHIC STYLE:** Use very short phrases (3-5 words) for bullets. No long sentences.
4. **CITATIONS:** Every single bullet MUST end with: ***(Source: Transcript ID, Turn No)***.

FORMAT:
## Causal Explanation
[1-sentence summary].

## Key Causal Factors
- [Short Phrase] ***(Source: ID, Turn)***
- [Short Phrase] ***(Source: ID, Turn)***
- [Short Phrase] ***(Source: ID, Turn)***
- [Short Phrase] ***(Source: ID, Turn)***

## Recommendations
1. **[Short Name]:** [Action] ***(Source: ID, Turn)***
2. **[Short Name]:** [Action] ***(Source: ID, Turn)***
3. **[Short Name]:** [Action] ***(Source: ID, Turn)***
4. **[Short Name]:** [Action] ***(Source: ID, Turn)***

[METADATA_START]
GROUNDING: [1 sentence on which pattern was most important]
CONFIDENCE: [High/Medium based on data match]
[METADATA_END]
"""

        try:
            response = self.client.models.generate_content(
                model='gemini-2.5-flash',
                contents=prompt
            )
            return response.text
        except Exception as e:
            return f"Error generating explanation: {e}"

try:
    API_KEY = userdata.get('GEMINI_API_KEY')
except:
    API_KEY = input("Enter Gemini API Key: ")

# Initialize System
system = CausalQuerySystem(
    turn_df=turn_df,
    conv_summary_df=conv_summary_df,
    collection=evidence_collection,
    api_key=API_KEY
)

TEST_QUERIES = [
    "Why do customers escalate to supervisors?",
    "What triggers fraud alert investigations?",
    "Why do customers complain about delivery status?",
    "What causes customers to call back multiple times for the same issue?",
    "Why are refund requests frequently delayed or denied?",
    "What agent behaviors lead to negative customer sentiment?",
    "What technical issues cause login or transaction failures?",
    "Why do identity verification procedures cause customer frustration?",
    "What patterns lead to customers threatening to close their accounts?",
    "What are the primary triggers for sudden spikes in customer anger?"
]

all_results = []

# Run Loop
print("\n" + "="*70)
print("STARTING BATCH PROCESSING")
print("="*70)


for i, q in enumerate(TEST_QUERIES):
    result = system.query(q)

    full_text = result['explanation']

    dynamic_category = system._get_category(q)

    metadata_match = re.search(r"\[METADATA_START\](.*?)\[METADATA_END\]", full_text, re.DOTALL)
    gemini_meta = metadata_match.group(1).strip() if metadata_match else "General analysis."

    clean_output = re.sub(r"\[METADATA_START\].*?\[METADATA_END\]", "", full_text, flags=re.DOTALL).strip()

    remarks_str = (
        f"🔍 {gemini_meta} | "
        f"Model: multi-qa-mpnet-base (768-dim) | "
        f"Evidence: {result['evidence_count']} turns from {len(result['retrieved_ids'])} transcripts | "
        f"Data Floor: -0.954 Sentiment."
    )

    all_results.append({
        "Query_Id": f"Q{i+9:03d}",
        "Query": q,
        "Query_Category": dynamic_category,
        "System_Output": clean_output,
        "Remarks": remarks_str
    })

    print("\n💡 SYSTEM OUTPUT PREVIEW:")
    print(result['explanation'][:500] + "...\n")

# Save to CSV
submission_df = pd.DataFrame(all_results)
submission_df.to_csv("task1_queries.csv", index=False)
print("✅ Saved 'task1_queries.csv' successfully!")

### Full Transcript Context Builder

This helper function retrieves the complete turn-by-turn dialogue for selected transcript IDs.
It assembles speaker-labeled conversation history in order, enabling deeper multi-turn reasoning and context-aware analysis.


In [ ]:
def get_full_transcript_text(transcript_ids, turn_df):
    """
    Pulls the complete dialogue history for specific transcript IDs.
    Used for Deep-Dive reasoning in Turn 2+.
    """
    context_text = ""
    for tid in transcript_ids:
        # Get all turns for this transcript, sorted
        chat_rows = turn_df[turn_df['transcript_id'] == tid].sort_values('turn_no')

        context_text += f"\n\n--- TRANSCRIPT {tid} ---\n"
        for _, row in chat_rows.iterrows():
            context_text += f"T{row['turn_no']} ({row['speaker']}): {row['text']}\n"

    return context_text

### Robust Multi-Turn Reasoning System

- This class enables multi-turn, session-based causal analysis over customer service transcripts.
- It locks relevant transcripts during the first query, preserves conversation history, and ensures all follow-up reasoning stays grounded in the same evidence across turns.

**Output:**
- `Task_2.csv` containing analysis-ready results for reporting and PDF export

In [ ]:
from google.colab import userdata
from google import genai
import pandas as pd
import time

class RobustMultiTurnSystem:
    def __init__(self, task1_engine):
        self.engine = task1_engine
        self.sessions = {}

        try:

            secret_key = userdata.get('GEMINI_API_KEY')
            if secret_key:
                print("API KEY")
                self.engine.client = genai.Client(api_key=secret_key)
            else:
                print("No APi key")
        except Exception as e:
            print(f"No api key")

    def start_session(self, session_id):
        self.sessions[session_id] = {
            'history': [],
            'evidence_lock': [],
            'full_context_cache': ""
        }
        print(f"Session {session_id}")



    def _get_category(self, user_query):
        prompt = f"""
        Classify this customer service query into ONE of these categories:
        - Escalation Analysis
        - Fraud & Security
        - Logistics & Delivery
        - Billing & Payments
        - General Support

        QUERY: "{user_query}"

        Output only the category name.
        """
        try:
            response = self.engine.client.models.generate_content(
                model='gemini-2.5-flash',
                contents=prompt
            )
            return response.text.strip()
        except:
            return "Causal Analysis"


    def query(self, user_query, session_id, turn_number):
        session = self.sessions[session_id]

        if turn_number == 1:
            print(f"Turn 1 Retrieval for: {user_query}")

            results = self.engine.query(user_query, top_k=5)

            found_ids = results['retrieved_ids']
            session['evidence_lock'] = found_ids

            session['full_context_cache'] = get_full_transcript_text(found_ids, self.engine.turn_df)

            response = results['explanation']

        else:
            print(f"Turn {turn_number} Reasoning (Locked to {len(session['evidence_lock'])} transcripts)")

            history_str = ""
            for turn in session['history']:
                history_str += f"User: {turn['query']}\nAI: {turn['response']}\n"

            prompt = f"""
            You are an expert analyst having a conversation about specific customer service transcripts.

            LOCKED EVIDENCE (Do not discuss other transcripts):
            {session['full_context_cache']}

            CONVERSATION HISTORY:
            {history_str}

            CURRENT QUESTION: "{user_query}"

            INSTRUCTIONS:
            1. Answer the question specifically using the LOCKED EVIDENCE provided above.
            2. Cite specific Turn Numbers (e.g., "In T12...") to prove your point.
            3. If the user asks for a comparison, compare the locked transcripts.
            4. Keep the response under 200 words.
            5. Inside the Reasoning you can add (Source: ID, Turn) there is 2 to 3 things must required
            6. Inside Evidence there is 5 to 6 minimum point which support the reasoing
            7. If you thing the (Source: ID, Turn) have nothing work don't add that
            8. ONLY REQUIRED THINK WHICH SATASIFY THE QUERY ONELY THAT THING YOU HAVE TO SUPPLY
            9. ***THE RULE IS STREET WHEN YOU WRITE (Source: ID, Turn) MAKE SURE ID IS 'XXXX-XXXX-XXXX-XXXX'***

            Output Format:
            ## Reasoning
            [Answer]
            ## Evidence
            - [Point 1] (Source: ID, Turn)
            - [Point 2] (Source: ID, Turn)
            """

            try:
                response = self.engine.client.models.generate_content(
                    model='gemini-2.5-flash',
                    contents=prompt
                ).text
            except Exception as e:
                response = f"Error in reasoning: {e}"

        session['history'].append({
            'query': user_query,
            'response': response
        })

        return response

In [ ]:
import time

secret_key = userdata.get('GEMINI_API_KEY')
if secret_key:
    print("API KEY")
else:
    print("No API KEY")

system = CausalQuerySystem(
    turn_df=turn_df,
    conv_summary_df=conv_summary_df,
    collection=evidence_collection,
    api_key=secret_key
)

task2_system = RobustMultiTurnSystem(system)

# 2. Define the Test Chains
TEST_CHAINS = [
    {
        "session_id": "S001",
        "topic": "Escalation",
        "turns": [
            "Why do customers escalate to supervisors?",
            "Did the agent try to de-escalate before the manager was requested?",
            "What was the specific sentiment score when the customer got angry?"
        ]
    },
    {
        "session_id": "S006",
        "turns": [
            "What product features did customers ask about most?",
            "Did agents provide accurate and complete product information?",
            "Were there any upselling or cross-selling attempts?"
         ]
    }
]

# 3 - Execution Loop
task2_results = []

for chain in TEST_CHAINS:
    sid = chain['session_id']
    task2_system.start_session(sid)

    for i, q in enumerate(chain['turns'], 1):
        print(f"\nProcessing {sid} - Turn {i}...")

        # Query
        output = task2_system.query(q, sid, turn_number=i)
        dynamic_Query_Category = task2_system._get_category(q)

        # Save Result
        task2_results.append({
            "Query_Id": f"Q{sid}_{chr(64 + i)}",
            "Query_Category": dynamic_Query_Category,
            "Query": q,
            "System_Output": output,
            "Remarks": str(task2_system.sessions[sid]['evidence_lock'])
        })

        time.sleep(20)

pd.DataFrame(task2_results).to_csv("task60_interactive_submission.csv", index=False)

In [ ]:
dataset6 = pd.read_csv("task60_interactive_submission.csv")
dataset5 = pd.read_csv("task50_interactive_submission.csv")
dataset4 = pd.read_csv("task40_interactive_submission.csv")
dataset3 = pd.read_csv("task30_interactive_submission.csv")
dataset2 = pd.read_csv("task20_interactive_submission.csv")
dataset1 = pd.read_csv("task10_interactive_submission.csv")
dataset7 = pd.read_csv("task70_interactive_submission.csv")

In [ ]:
final_dataset = pd.concat([
    dataset1,
    dataset2,
    dataset3,
    dataset4,
    dataset5,
    dataset6,
    dataset7
], ignore_index=True)

In [ ]:
df_sorted = final_dataset.sort_values(by='Query_Id', ascending=True)

In [ ]:
df_sorted.to_csv("Task_2.csv", index=False)

**END**